In [3]:
import sys
sys.path.append('/srv/public/Maikel/FiniteVolumeSolver/extra/')

import yt
import os
import numpy as np
import matplotlib.pyplot as plt
import amrex.plotfiles as da

valueString = 'Pressure' # which value we want to plot
# possible values are: ['Density', 'Energy', 'Momentum_0', 'Momentum_1', 'Pressure', 'Species_0', 'Species_1', 'SpeedOfSound', 'vfrac']

dataPath = "/srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary/"
plenum_number = 0
MASK_DATA_WITH_VFRAC = True
vfrac_cutoff = 1.0e-16

def PrintProgress(i, plotfiles):
  ny = len(plotfiles)
  progress = int(100.0 * float(i) / (ny - 1))
  print('[{:3d}%] Reading plotfile {}'.format(progress, plotfiles[i]))

yt.funcs.mylog.setLevel(50)
output_dir = dataPath+'/Plotfiles/Plenum%i'%(plenum_number)

dirs = os.listdir(output_dir)
plotfiles = ['{}/{}'.format(output_dir, plt) for plt in dirs]
list.sort(plotfiles)
#plotfiles = plotfiles[::10]

output_path = '{}/Visualization'.format(dataPath)
os.makedirs(output_path, exist_ok=True)

%matplotlib inline
for i, plotfile in enumerate(plotfiles):
   PrintProgress(i, plotfiles)
   (p, rho, rhou, rhov, c, vols), current_time, extent = da.load(plotfile, ["Pressure", "Density", "Momentum_0", "Momentum_1", "SpeedOfSound", 'vfrac'])
   f, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 15))
   f.suptitle('Time = {:.2f}'.format(current_time))
   # pressure image
   im_p = axs[0].imshow(p, origin='lower', vmin=1.0, vmax=1.7, interpolation='gaussian', extent=extent, cmap='tab20c')
   axs[0].set_title('Pressure')
   plt.colorbar(im_p, ax=axs[0])
   # temperature image
   T = p / rho
   im_T = axs[1].imshow(T, origin='lower', vmin=1.0, vmax=2.0, interpolation='none', extent=extent)
   axs[1].set_title('Temperature')
   plt.colorbar(im_T, ax=axs[1])
   # velocity field
   u = np.where(vols > 1e-14, rhou / rho, np.nan)
   v = np.where(vols > 1e-14, rhov / rho, np.nan)
   skip = 5
   scale = 0.2
   axs[2].quiver(u[::skip,::skip], v[::skip,::skip], scale=scale, units='xy')
   axs[2].set_title('Velocity Field')
   f.savefig('{}/Figure{:04d}.png'.format(output_path, i))
   f.clear()
   plt.close(f)

[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000000
[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000016
[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000032
[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000049
[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000066
[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000083
[  0%] Reading plotfile /srv/public/Maikel/FiniteVolumeSolver/build_2D-Release/SEC_Plenum_TurbineBoundary//Plotfiles/Plenum0/plt000000100
[  0%] Reading plotfile /srv/publi